## Basic mechanic

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
#tf.enable_eager_execution()
sess = tf.Session()

### Dataset structure

In [4]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))
print(dataset1.output_types)  # ==> "tf.float32"
print(dataset1.output_shapes)  # ==> "(10,)"

dataset2 = tf.data.Dataset.from_tensor_slices(
   {"a": tf.random_uniform([4]),
    "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)})
print(dataset2.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(dataset2.output_shapes) 
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
print(dataset3.output_types)  # ==> (tf.float32, (tf.float32, tf.int32))
print(dataset3.output_shapes) 

<dtype: 'float32'>
(10,)
{'a': tf.float32, 'b': tf.int32}
{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}
(tf.float32, {'a': tf.float32, 'b': tf.int32})
(TensorShape([Dimension(10)]), {'a': TensorShape([]), 'b': TensorShape([Dimension(100)])})


### Creating an iterator
#### one-shot
the simplest form of iterator, which only supports iterating once through a dataset, with no need for explicit initialization. One-shot iterators handle almost all of the cases that the existing queue-based input pipelines support, but they do not support parameterization.

In [13]:
dataset = tf.data.Dataset.range(100)
iterator = dataset.make_one_shot_iterator()
for i in range(100):
    value =  iterator.get_next()
    assert i == value.numpy()

#### initializable
run an explicit iterator.initializer operation before using it. it enables you to parameterize the definition of the dataset, using one or more tf.placeholder() tensors that can be fed when you initialize the iterator

In [7]:
max_value = tf.placeholder(tf.int64, shape = [])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
sess.run(iterator.initializer, feed_dict = {max_value: 10})
for i in range(10):
    value = sess.run(next_element)
    assert i == value

#### reinitializable
can be initialized from multiple different Dataset objects.

In [9]:
training_dataset = tf.data.Dataset.range(20).map(
lambda x: x + tf.random_uniform([], -5, 5, tf.int64))
validation_dataset = tf.data.Dataset.range(10)
iterator = tf.data.Iterator.from_structure(training_dataset.output_types, 
                                          training_dataset.output_shapes)
next_element = iterator.get_next()
training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)
for _ in range(2):
    sess.run(training_init_op)
    for _ in range(20):
        sess.run(next_element)
    sess.run(validation_init_op)
    for _ in range(10):
        sess.run(next_element)

#### feedable
can be used together with tf.placeholder to select what Iterator to use in each call to tf.Session.run, via the familiar feed_dict mechanism. It offers the same functionality as a reinitializable iterator, but it does not require you to initialize the iterator from the start of a dataset when you switch between iterators.

In [ ]:
training_dataset = tf.data.Dataset.range(20).map(
lambda x: x + tf.random_uniform([], -5, 5, tf.int64))
validation_dataset = tf.data.Dataset.range(10)
handle = tf.placeholder(tf.string, shape = [])
iterator = tf.data.Iterator.from_string_handle(
handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()
training_handle = sess.run(training_iterator)
validation_handle = sess.run(validation_iterator)
while True:
    for _ in range(10):
        sess.run(next_element, feed_dict={handle:training_handle})
    sess.run(validation_iterator.initializer)
    for _ in range(5):
        sess.run(next_element, feed_dict = {handle: validation_handle})

In [11]:
dataset = tf.data.Dataset.range(5)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
result = tf.add(next_element, next_element)
sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(result))
    except tf.errors.OutOfRangeError:
        break 

0
2
4
6
8


### Saving iterator state
The `tf.contrib.data.make_saveable_from_iterator` function creates a SaveableObject from an iterator, which can be used to save and restore the current state of the iterator. A saveable object thus created can be added to `tf.train.Saver` variables list or the` tf.GraphKeys.SAVEABLE_OBJECTS` collection for saving and restoring in the same manner as a `tf.Variable`

In [ ]:
# Create saveable object from iterator.
saveable = tf.contrib.data.make_saveable_from_iterator(iterator)
# Save the iterator state by adding it to the saveable objects collection.
tf.add_to_collection(tf.GraphKeys.SAVEABLE_OBJECTS, saveable)

## Reading input data
### Consuming NumPy arrays

In [ ]:
with np.load("/var/data/training_data.npy") as data:
    features = data["features"]
    labels = data["labels"] 
# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]
features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
iterator = dataset.make_initializable_iterator()
sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})

### Consuming TFRecord data

In [ ]:
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)  # Parse the record into tensors.
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(32)
iterator = dataset.make_initializable_iterator()
# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.
# Initialize `iterator` with training data.
training_filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
sess.run(iterator.initializer, feed_dict={filenames: training_filenames})
# Initialize `iterator` with validation data.
validation_filenames = ["/var/data/validation1.tfrecord", ...]
sess.run(iterator.initializer, feed_dict={filenames: validation_filenames})

### Consuming text data

In [ ]:
filenames = ["/var/data/file1.txt", "/var/data/file2.txt"]
dataset = tf.data.Dataset.from_tensor_slices(filenames)
# Use `Dataset.flat_map()` to transform each file as a separate nested dataset,
# and then concatenate their contents sequentially into a single "flat" dataset.
# * Skip the first line (header row).
# * Filter out lines beginning with "#" (comments).
dataset = dataset.flat_map(
    lambda filename: (
        tf.data.TextLineDataset(filename)
        .skip(1)
        .filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#"))))

### Consuming CSV data

In [ ]:
# Creates a dataset that reads all of the records from two CSV files, each with
# eight float columns
filenames = ["/var/data/file1.csv", "/var/data/file2.csv"]
record_defaults = [tf.float32] * 8   # Eight required float columns
dataset = tf.contrib.data.CsvDataset(filenames, record_defaults)
# Creates a dataset that reads all of the records from two CSV files, each with
# four float columns which may have missing values
record_defaults = [[0.0]] * 8
dataset = tf.contrib.data.CsvDataset(filenames, record_defaults)
# Creates a dataset that reads all of the records from two CSV files with
# headers, extracting float data from columns 2 and 4.
record_defaults = [[0.0]] * 2  # Only provide defaults for the selected columns
dataset = tf.contrib.data.CsvDataset(filenames, record_defaults, header=True, select_cols=[2,4])

## Preprocessing data with `Dataset.map()`
The `Dataset.map(f)` transformation produces a new dataset by applying a given function f to each element of the input dataset.

In [ ]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)
    image_resized = tf.image.resize_images(image_decoded, [28, 28])
    return image_resized, label
# A vector of filenames.
filenames = tf.constant(["/var/data/image1.jpg", "/var/data/image2.jpg", ...])
# `labels[i]` is the label for the image in `filenames[i].
labels = tf.constant([0, 37, ...])
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(_parse_function)

### Batching dataset elements

In [13]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
dataset = dataset.padded_batch(4, padded_shapes=[None])
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
print(sess.run(next_element))  # ==> [[0, 0, 0], [1, 0, 0], [2, 2, 0], [3, 3, 3]]
#print(sess.run(next_element))

[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]


## Training workflows
### Processing multiple epochs
The simplest way to iterate over a dataset in multiple epochs is to use the `Dataset.repeat()` transformation. 
### Randomly shuffling input data
The `Dataset.shuffle()` transformation randomly shuffles the input dataset using a similar algorithm to tf.RandomShuffleQueue: it maintains a fixed-size buffer and chooses the next element uniformly at random from that buffer.
### Using high-level APIs
`MonitoredTrainingSession` uses the `tf.errors.OutOfRangeError` to signal that training has completed, so to use it with the tf.data API, we recommend using `Dataset.make_one_shot_iterator()`.

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(32)
dataset = dataset.repeat(num_epochs)
iterator = dataset.make_one_shot_iterator()

next_example, next_label = iterator.get_next()
loss = model_function(next_example, next_label)

training_op = tf.train.AdagradOptimizer(...).minimize(loss)

with tf.train.MonitoredTrainingSession(...) as sess:
    while not sess.should_stop():
        sess.run(training_op)